In [11]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import requests
from IPython.display import display, HTML
from ipywidgets import interact_manual, Button, Output
import json
import warnings

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

In [16]:
def geocode(location):
    query_string = {'q' : location, 'format': 'json'}
    url = 'https://nominatim.openstreetmap.org/search'
    response = requests.get(url, params = query_string)
    response.raise_for_status()
    geodata = response.json()
    return geodata

def GetOffenders(lat, lon):
    apikey = '3CA27D62-D057-4D9B-BF1F-F22C2906FE51'
    lite = 0
    miles = 1
    type = 'searchbyspecificlatlong'
    url = 'https://services.familywatchdog.us/rest/json.asp?'
    query_string = {'key' : apikey, 'lite' : lite, 'type' : type, 'lat' : lat, 'long' : lon, 'miles' : miles }
    response = requests.get(url, params = query_string)
    response.raise_for_status()
    dataset = response.json()
    return dataset

fofo = GetOffenders(40.7127281, -74.0060152)

def write_json(data, filename):
    with open(filename, 'w') as f:
        file = json.dump(data, f, indent=4)
    return file

def getOffenderInfo(file):
    new_dict = []
    myjson = open(file, 'r')   #citation; recieved tons of guidance on extracting data from a json file here: https://www.youtube.com/watch?v=aj4L7U7alNU
    jsondata = myjson.read()
    obj = json.loads(jsondata)
    list= (obj['offenders'])
    for off in list:
        if off['convictions'] != []:
            off_charges = off['convictions'][0]['charge']
        else: 
            off_charges = []
        if off['markings'] != []:
            off_markings = off['markings'][0]['description']
        else:
            off_markings = []
        name = off['name']
        dob = off['dob']
        sex = off['sex']
        race = off['race']
        hair = off['hair']
        height = off['height']
        weight = off['weight']
        convictlevel = off['convictiontype']
        homeaddress = off['street1'], off['city'], off['state'], off['zipcode']
        photo = off['photo']
        latitude = off['latitude']
        longitude = off['longitude']
        new_dict.append({'name': name, 'DOB': dob, 'sex': sex, 'race': race, 'hair': hair, 'height': height, 'weight': weight, 'convictlevel': convictlevel, 'homeaddress': homeaddress,'charges': off_charges, 'markings': off_markings, 'photo': photo, 'latitude': latitude, 'longitude': longitude})
        with open('offenderinformation_data.json', 'w') as f:
            newfile = json.dump(new_dict, f, indent=4)
    return newfile
    

display(HTML('<h2>*** Run for a saved dataset of offenders within your city ***</h2>'))
@interact_manual(location="")
def GetLocation(location):
    loco = geocode(location)
    for coord in loco:
        lat = coord['lat']
        lon = coord['lon']
        fofo = GetOffenders(lat, lon)
        beo = write_json(fofo, 'original_offenders_dataset.json')
        ric = getOffenderInfo('original_offenders_dataset.json')
        return ric

/var/folders/8p/cg1w2pt11vl28hbvp8_dl9jm0000gn/T/ipykernel_40328/2958388532.py:62: DeprecationWarning: on_submit is deprecated. Instead, set the .continuous_update attribute to False and observe the value changing with: mywidget.observe(callback, 'value').
  @interact_manual(location="")


interactive(children=(Text(value='', description='location'), Button(description='Run Interact', style=ButtonS…

In [17]:
# In[ ]:

import pandas as pd
# citation; recieved guidance on json pandas dataframe from https://www.youtube.com/watch?v=RRSJjxJhVEM&t=352s

off_file = 'offenderinformation_data.json'
with open(off_file) as f:
    data = json.load(f)
    
all_offenders = pd.DataFrame(columns=['Name', 'DOB', 'Sex', 'Race', 'Hair', 'Height', 'Weight', 'Conviction Level', 'Home Address', 'Charge(s)', 'Marking(s)', 'Photo', 'Latitude', 'Longitude'])

for i in range(0, len(data)):
    currentitem = data[i]
    all_offenders.loc[i] = [data[i]['name'], data[i]['DOB'], data[i]['sex'], data[i]['race'], data[i]['hair'], data[i]['height'], data[i]['weight'], data[i]['convictlevel'], data[i]['homeaddress'], data[i]['charges'], data[i]['markings'], data[i]['photo'], data[i]['latitude'], data[i]['longitude']]

display(HTML('<h2>************************************ A dataframe of each offender ************************************</h2>'))    
    
all_offenders    

,Name,DOB,Sex,Race,Hair,Height,Weight,Conviction Level,Home Address,Charge(s),Marking(s),Photo,Latitude,Longitude
0,MANOJ KUMAR SAXENA,06/07/1970,M,Asian or Pac. Isl.,Black,5ft 04in,138,3,"[400 JAMES ST, SYRACUSE, NY, 13203]","53a-71 - Second Degree Sexual Assault, other t...",[],http://photo.familywatchdog.us/OffenderPhoto/O...,43.0525216,-76.1475576
1,Terry Olin Taft,04/29/1964,Male,White,Grey,5ft 11in,200,3,"[Last Reported Address - Out of State, syracus...","SEX OFFENSE, OTHER STATE (14-27.5A(A) -- Sexua...",Tattoo - Left Arm,http://photo.familywatchdog.us/OffenderPhoto/O...,43.0481,-76.1474
2,William Craig Cothran,01/02/1981,Male,White,Brown,5ft 10in,190,1,"[Last Reported Address - Out of State, Syracus...",Sx Offender fail to not. Sheriff; F.S. 943.043...,Tattoo - Left Hand,http://photo.familywatchdog.us/OffenderPhoto/O...,43.05489,-76.15033
3,Angel Luis Velez,01/08/1967,Male,White,Black,5ft 06in,180,1,"[Last Reported Address - Out of State, Syracus...","SEX OFFENSE, OTHER STATE (SEXUAL MISCONDUCT: P...",Tattoo - Left Arm,http://photo.familywatchdog.us/OffenderPhoto/O...,43.05489,-76.15033
4,Robert M Cornwell Jr,07/13/1962,Male,White,Unknown/bald,6ft 02in,215,,"[Last Reported Address - Out of State, Syracus...",[],[],http://photo.familywatchdog.us/OffenderPhoto/O...,43.05489,-76.15033
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
323,ERVIN WILLIAMS,10/18/1966,Male,Black,Black,5ft 09in,148,4,"[CATHOLIC CHARITIES SHELTER, SYRACUSE, NY, 13210]",Rape-1st:Forcible Compulsion,"Tattoo-Shoulder, left",http://photo.familywatchdog.us/OffenderPhoto/O...,43.04689,-76.13454
324,JUSTIN ARNOLD,6/14/1963,Male,White,Gray,5ft 10in,220,5,"[400 JAMES ST, SYRACUSE, NY, 13203]",Sexual Abuse 1st: Contact W/Victim Less Than 1...,None Reported,http://photo.familywatchdog.us/OffenderPhoto/O...,43.05273,-76.14784
325,EDWARD MARK SAGOR,10/1/1957,Male,White,Brown,5FT 9IN,170,1,"[511 S STATE ST, SYRACUSE, NY, 13202]",Sexual abuse in the second degree. A person ...,"Forearm, left - Grim Ripper with flower",http://photo.familywatchdog.us/OffenderPhoto/O...,43.0467622,-76.1469716
326,"SCHENK JR, WILLIAM STANLEY",05/23/1962,MALE,WHITE,BROWN,5ft 10in,165,1,"[120 GIFFORD ST, SYRACUSE, NY, 13202]",SEXUAL ABUSE 1ST DEGREE,"SCAR, LEG, LEFT",http://photo.familywatchdog.us/OffenderPhoto/O...,43.043853,-76.154862


In [18]:
# In[ ]:


all_offenders['SexCat'] = np.nan
    
all_offenders['SexCat'][ all_offenders['Sex'] == 'Male'] = 'Male'
all_offenders['SexCat'][ all_offenders['Sex'] == 'M'] = 'Male'
all_offenders['SexCat'][ all_offenders['Sex'] == 'MALE'] = 'Male'
all_offenders['SexCat'][ all_offenders['Sex'] == 'Female'] = 'Female'

all_offenders['SexCat'].value_counts()

risk1 = all_offenders[ all_offenders['Conviction Level'] == '1']
risk1['SexCat'].value_counts()

risk2 = all_offenders[ all_offenders['Conviction Level'] == '2']
risk2['SexCat'].value_counts()

risk3 = all_offenders[ all_offenders['Conviction Level'] == '3']
risk3['SexCat'].value_counts()

risk4 = all_offenders[ all_offenders['Conviction Level'] == '4']
risk4['SexCat'].value_counts()

sex_series = all_offenders['SexCat'].value_counts()
risk1_series = risk1['SexCat'].value_counts()
risk2_series = risk2['SexCat'].value_counts()
risk3_series = risk3['SexCat'].value_counts()
risk4_series = risk4['SexCat'].value_counts()

sex_df = pd.DataFrame( {'All' :sex_series, 'Risk Level 1' :risk1_series, 'Risk Level 2' :risk2_series,'Risk Level 3' :risk3_series,'Risk Level 4' :risk4_series})

display(HTML('<h2>***************** How do female and male offenders compare on their risk level? *****************</h2>'))

sex_df

/var/folders/8p/cg1w2pt11vl28hbvp8_dl9jm0000gn/T/ipykernel_40328/1720195448.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_offenders['SexCat'][ all_offenders['Sex'] == 'Male'] = 'Male'


,All,Risk Level 1,Risk Level 2,Risk Level 3,Risk Level 4
SexCat,,,,,
Female,3,2,NaN,NaN,NaN
Male,325,142,NaN,6.0,45.0


In [5]:
# In[ ]:


all_offenders['RaceCat'] = np.nan
    
all_offenders['RaceCat'][ all_offenders['Race'] == 'White'] = 'White'
all_offenders['RaceCat'][ all_offenders['Race'] == 'W'] = 'White'
all_offenders['RaceCat'][ all_offenders['Race'] == 'Black'] = 'Black'
all_offenders['RaceCat'][ all_offenders['Race'] == 'Asian'] = 'Asian'

risk_1 = all_offenders[ all_offenders['Conviction Level'] == '1']
risk_1['RaceCat'].value_counts()

risk_2 = all_offenders[ all_offenders['Conviction Level'] == '2']
risk_2['RaceCat'].value_counts()

risk_3 = all_offenders[ all_offenders['Conviction Level'] == '3']
risk_3['RaceCat'].value_counts()

risk_4 = all_offenders[ all_offenders['Conviction Level'] == '4']
risk_4['RaceCat'].value_counts()

race_series = all_offenders['RaceCat'].value_counts()
risk1_seq = risk_1['RaceCat'].value_counts()
risk2_seq = risk_2['RaceCat'].value_counts()
risk3_seq = risk_3['RaceCat'].value_counts()
risk4_seq = risk_4['RaceCat'].value_counts()

race_df = pd.DataFrame( {'All' :race_series, 'Risk Level 1' :risk1_seq, 'Risk Level 2' :risk2_seq,'Risk Level 3' :risk3_seq,'Risk Level 4' :risk4_seq})

display(HTML('<h2>************* How does race/ethnic background come into play for conviction level? *************</h2>'))

race_df

/var/folders/8p/cg1w2pt11vl28hbvp8_dl9jm0000gn/T/ipykernel_40328/2201395829.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_offenders['RaceCat'][ all_offenders['Race'] == 'White'] = 'White'


,All,Risk Level 1,Risk Level 2,Risk Level 3,Risk Level 4
RaceCat,,,,,
Asian,7,3,NaN,NaN,1
Black,191,99,1.0,4.0,45
White,232,96,NaN,7.0,22


In [21]:
pip install folium

Note: you may need to restart the kernel to use updated packages.


In [22]:
!pip install folium

In [23]:
# In[ ]:

import pandas as pd
import folium

#citation; recieved guidance with this part of the program from https://www.youtube.com/watch?v=FdqDgoG-SFM&t=891s

def set_marker_color(row):
    if row['Conviction Level'] == '1':
        return 'green'
    elif row['Conviction Level'] == '2':
        return 'blue'
    elif row['Conviction Level'] == '3':
        return 'pink'
    elif row['Conviction Level'] == '4':
        return 'purple'
    
all_offenders['color'] = all_offenders.apply(set_marker_color, axis=1)

CENTER_US = (39.8333333,-98.585522)
off_map = folium.Map(
    location=CENTER_US,
    zoom_start=4
)

for _, indiv in all_offenders.iterrows():
    folium.Marker(
        location=[indiv['Latitude'], indiv['Longitude']],
        popup=(indiv['Name'], indiv['Charge(s)'], indiv['Home Address']),
        tooltip=(indiv['Name'], indiv['Charge(s)'], indiv['Home Address']),
        icon=folium.Icon(color=indiv['color'], prefix='fa', icon='circle'),
    ).add_to(off_map)
    
    
display(HTML('<h2>**************************************** Offenders in your area ****************************************</h2>'))    
display(HTML('<body> Risk Level 1: green </body>'))
display(HTML('<body> Risk Level 2: blue </body>'))
display(HTML('<body> Risk Level 3: pink </body>'))
display(HTML('<body> Risk Level 4: purple </body>'))
            
    
off_map.save('nearest_offenders_in_the_area.html')

off_map

/var/folders/8p/cg1w2pt11vl28hbvp8_dl9jm0000gn/T/ipykernel_40328/236258749.py:31: UserWarning: color argument of Icon should be one of: {'darkred', 'white', 'darkblue', 'purple', 'red', 'darkpurple', 'pink', 'orange', 'lightgreen', 'lightred', 'darkgreen', 'black', 'beige', 'green', 'blue', 'cadetblue', 'lightgray', 'gray', 'lightblue'}.
  icon=folium.Icon(color=indiv['color'], prefix='fa', icon='circle'),
